In [1]:
#preprocess ml
#train
#test

In [14]:
import pandas as pd
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score,accuracy_score,roc_auc_score
from sklearn.ensemble import GradientBoostingRegressor,ExtraTreesClassifier #随机森林分类模型
import numpy as np
import os

In [18]:
symbol = 'bc'
for m in range(10,20,10):
    print(m/60,m/(24*60),'day')
    test_size = 0.4
    train_size = 1 - test_size
    df = []
    csv_filename = f"./{symbol}_1min.csv"  # Replace with the actual filename
    df = pd.read_csv(csv_filename)
    df.loc[:, 'open':'hold'] = df.loc[:, 'open':'hold'].apply(pd.to_numeric, errors='coerce')
    df = df.dropna()
    #df.loc[:,'open':'close'] = df.loc[:,'open': 'close'].astype(float)
    # 处理时间间隔
    df['datetime'] = pd.to_datetime(df['datetime'])
    df["(t+1)-(t)"] = np.nan
    time_threshold_window = pd.DateOffset(minutes=m)
     ######-----------计算label
    for index, row in df.iterrows():
        target_time = row['datetime'] + time_threshold_window
        result_row = df.loc[df['datetime'] == target_time]
        if not result_row.empty:
            #print(result_row['close'].iloc[0])
            df.loc[index, "(t+1)-(t)"] = result_row['close'].iloc[0] - row['close']
    df = df.dropna()
    #print(df)
    df['label'] = 0
    # 构建对应表
    df = df.dropna()
    ##去除异常
    df.drop(df[abs(df["(t+1)-(t)"]) > 10000].index, inplace=True)
    ##
    ################
    sigma = np.std(df["(t+1)-(t)"][: int(train_size*len(df))])
    mean = np.mean(df["(t+1)-(t)"][: int(train_size*len(df))])
    df['label'] = df["(t+1)-(t)"].apply(lambda x: 1 if x > mean + 3 * sigma else (-1 if x < mean - 3 * sigma else 0))
    df = df.dropna()
    #####================
    os.makedirs(f'{symbol}', exist_ok=True)
    df.to_csv(f'./{symbol}/{symbol}{m}_ans.csv',mode='w')
    #time_threshold


    class RF_predict:
        def __init__(self, _df):
            self.tsData = _df
            self.test_data = []
        def make_predict(self, size ,test_data):
            #node = test_size
            node = int(len(self.tsData) *size)
            #
            #print(self.tsData)
            # 构建数据集
            train_data = self.tsData[: len(self.tsData) - int(node)]
            self.test_data = self.tsData[len(self.tsData) - node : ]
            train_X = train_data.loc[:, 'open': 'hold'].values
            train_y = train_data['label'].values

            test_X = self.test_data.loc[:, 'open': 'hold'].values
            test_y = self.test_data['label'].values
    
            # 进行预测
            clf = ExtraTreesClassifier()
            clf.fit(train_X, train_y)
            print(accuracy_score(train_y, clf.predict(train_X)))
            print(accuracy_score(test_y, clf.predict(test_X)),'\n')
            #print(roc_auc_score(test_y,clf.predict(test_X))) # 召回率
            print()

    a = RF_predict(df)
    a.make_predict(test_size,[])
    a.test_data

0.16666666666666666 0.006944444444444444 day
1.0
0.9982978723404256 




# 

0.16666666666666666 0.006944444444444444
1.0
0.9668585592068442 

0.3333333333333333 0.013888888888888888
1.0
0.9682809909950271 

0.5 0.020833333333333332
1.0
0.9696535363020807 

0.6666666666666666 0.027777777777777776
1.0
0.9697384261792711 

0.8333333333333334 0.034722222222222224
1.0
0.9687919054004632 

1.0 0.041666666666666664
1.0
0.9685784851811197 

1.1666666666666667 0.04861111111111111
1.0
0.9667888398533921 

1.3333333333333333 0.05555555555555555
1.0
0.9649697168112621 

1.5 0.0625
1.0
0.9628192401407815 

1.6666666666666667 0.06944444444444445
1.0
0.9618198216254134 

1.8333333333333333 0.0763888888888889
1.0
0.9642537980339589 

2.0 0.08333333333333333
1.0
0.9637791286727457 

2.1666666666666665 0.09027777777777778
1.0
0.9633840052875082 

2.3333333333333335 0.09722222222222222
1.0
0.9662171555679132 

2.5 0.10416666666666667
1.0
0.9661515820456218 

2.6666666666666665 0.1111111111111111
1.0
0.965458178703968 

2.8333333333333335 0.11805555555555555
1.0
0.962942704772323

ValueError: Found array with 0 sample(s) (shape=(0, 6)) while a minimum of 1 is required by ExtraTreesClassifier.